In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2023-07-09 18:40:47.811089: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-09 18:40:47.882561: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Filter out corrupted images

import os

num_skipped = 0
for folder_name in ("Apple___Apple_scab","Apple___Black_rot","Apple___Cedar_apple_rust","Apple___healthy",
                    "Background_without_leaves",
                    "Blueberry___healthy",
                    "Cherry___healthy","Cherry___Powdery_mildew",
                    "Corn___Cercospora_leaf_spot_Gray_leaf_spot","Corn___Common_rust","Corn___healthy","Corn___Northern_Leaf_Blight",
                    "Grape___Black_rot","Grape___Esca_Black_Measles","Grape___healthy","Grape___Leaf_blight_Isariopsis_Leaf_Spot",
                    "Orange___Haunglongbing_Citrus_greening",
                    "Peach___Bacterial_spot","Peach___healthy",
                    "Pepper___bell_Bacterial_spot", "Pepper___bell_healthy", 
                    "Potato___Early_blight","Potato___healthy", "Potato___Late_blight",
                    "Raspberry___healthy",
                    "Soybean___healthy", 
                    "Squash___Powdery_mildew",
                    "Strawberry___healthy","Strawberry___Leaf_scorch",
                    "Tomato___Target_Spot", "Tomato___Tomato_mosaic_virus",
                   "Tomato___Tomato_Yellow_Leaf_Curl_Virus", "Tomato___Bacterial_spot", "Tomato___Early_blight",
                   "Tomato___healthy", "Tomato___Late_blight", "Tomato___Leaf_Mold", "Tomato___Septoria_leaf_spot",
                   "Tomato___Spider_mites_Two_spotted_spider_mite" ):
    folder_path = os.path.join("dataset/without_augmentation", folder_name)
    print(folder_path)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

In [ ]:
image_size = (256, 256)
batch_size = 64

train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/without_augmentation",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical'
)

In [ ]:
# Using k-fold cross-validation

image_size = (256, 256)
batch_size = 64

ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/without_augmentation",
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical',
    seed=1337
)

k = 4
i = 1

# Split the dataset into k folds using tf.data.Dataset methods
dataset_size = len(ds)
print('Dataset size')
print(dataset_size)
fold_size = dataset_size // k
print('Fold size')
print(fold_size)

train_ds = []
val_ds = []

start = i * fold_size
print(start)
end = (i + 1) * fold_size
print(end)

# Create training and validation datasets for the current fold
val_ds = ds.skip(start).take(fold_size)
train_ds = ds.take(start).concatenate(ds.skip(end))

print('Val size')
print(val_ds.cardinality())
print('Train size')
print(train_ds.cardinality())


In [ ]:
# Create the dataset arrays using the 3-datasets approach

image_size = (256, 256)
batch_size = 32

train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/without_augmentation",
    validation_split=0.3,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical'
)

val_batches = val_ds.cardinality()
print(val_batches)
test_ds = val_ds.take((1*val_batches) // 3)
val_ds = val_ds.skip((1*val_batches) // 3)
print(train_ds.cardinality())
print(test_ds.cardinality())
print(val_ds.cardinality())


In [ ]:
# Print quantity of images in each dataset to check

train_ds_images = tf.data.experimental.cardinality(train_ds).numpy() * batch_size

print("Quantity of images of train ds:", train_ds_images)

train_ds_images = 0
for batch in train_ds:
    train_ds_images += batch[0].shape[0]

print("Quantity of images in train_ds:", train_ds_images)

val_ds_images = 0
for batch in val_ds:
    val_ds_images += batch[0].shape[0]

print("Quantity of images in val_ds:", val_ds_images)

test_ds_images = 0
for batch in test_ds:
    test_ds_images += batch[0].shape[0]

print("Quantity of images in test_ds:", test_ds_images)


In [ ]:
# Print the classes' names

class_names = train_ds.class_names
print("Class names:", class_names)

In [ ]:
# One-hot encoding

train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, 39)))
val_ds = val_ds.map(lambda x, y: (x, tf.one_hot(y, 39)))

In [ ]:
train_ds_images = tf.data.experimental.cardinality(train_ds).numpy() * batch_size

print("Quantity of images of train ds:", train_ds_images)

val_ds_images = tf.data.experimental.cardinality(val_ds).numpy() * batch_size

print("Quantity of images of val ds:", val_ds_images)

train_ds_images = 0
for batch in train_ds:
    train_ds_images += batch[0].shape[0]

print("Quantity of images in train_ds:", train_ds_images)

val_ds_images = 0
for batch in val_ds:
    val_ds_images += batch[0].shape[0]

print("Quantity of images in val_ds:", val_ds_images)

In [ ]:
# Example of train dataset item

print(train_ds.take(1))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in val_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        #plt.title(int(labels[i]))
        plt.axis("off")

In [ ]:
# Data augmentation definition

data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomBrightness(0.3),
        layers.RandomZoom(0.4),
        layers.RandomContrast(0.3),
    ]
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

In [ ]:
# Apply `data_augmentation` to the training images.
augmented_train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf.data.AUTOTUNE,
)

# Count the number of original and augmented images in the datasets.
original_count = tf.data.experimental.cardinality(train_ds).numpy()
augmented_count = tf.data.experimental.cardinality(augmented_train_ds).numpy()

# Calculate the number of augmented images.
num_augmented_images = augmented_count - original_count

print("Number of original images:", original_count)
print("Number of augmented count:", augmented_count)
print("Number of augmented images:", num_augmented_images)

# Prefetching samples in GPU memory helps maximize GPU utilization.

prefetched_train_ds = augmented_train_ds.prefetch(tf.data.AUTOTUNE)
prefetched_val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
# GPU-augmented - Prefetching samples in GPU memory helps maximize GPU utilization.

data_augmentation_test = keras.Sequential(
    [
       layers.RandomRotation(0), 
    ]
)

train_ds = train_ds.map(
    lambda img, label: (data_augmentation_test(img), label),
    num_parallel_calls=tf.data.AUTOTUNE,
)

prefetched_train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
prefetched_val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # x = data_augmentation(inputs)

    # Entry block
    # x = layers.Rescaling(1.0 / 255)(x)
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(64, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)

    activation = "softmax"
    units = num_classes
    

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=39)
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# Normal training

import pickle
import matplotlib.pyplot as plt

epochs = 100

callbacks = [
    tf.keras.callbacks.History(),
    keras.callbacks.ModelCheckpoint("3datasets_1e-3_manyGpuAug_stride2_batch32_64filters_categorical/{epoch}.keras")
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
history = model.fit(
    prefetched_train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=prefetched_val_ds,
)

with open("3datasets_1e-3_manyGpuAug_stride2_batch32_64filters_categorical/training_history.pkl", 'wb') as file:
    pickle.dump(history.history, file)
    
# summarize history for accuracy
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig("3datasets_1e-3_manyGpuAug_stride2_batch32_64filters_categorical/accuracy_plot.png")
# summarize history for loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig("3datasets_1e-3_manyGpuAug_stride2_batch32_64filters_categorical/loss_plot.png")

In [ ]:
# Normal training with k-fold

for fold, (train_index, val_index) in enumerate(kf.split(ds)):
    print(f"Fold: {fold+1}")
    
    # Create training and validation datasets for this fold
    train_ds = ds.take(train_index)
    val_ds = ds.take(val_index)

    # Reset the model and compile it
    model = create_model()  # Replace with your model creation code
    model.compile(...)  # Replace with your model compilation code

    # Train the model on the training dataset
    model.fit(train_ds, ...)

    # Evaluate the model on the validation dataset
    model.evaluate(val_ds, ...)

In [ ]:
# Load interrupted training
import pickle
import matplotlib.pyplot as plt

filePath = "1e-2_manyGpuAug_stride2_batch64_128filters-200e/100.keras"

interruptedModel = keras.models.load_model(filePath)

epochs = 100

callbacks = [
    tf.keras.callbacks.History(),
    keras.callbacks.ModelCheckpoint("1e-2_manyGpuAug_stride2_batch64_128filters-200e/{epoch}.keras"),
]

history = interruptedModel.fit(
    prefetched_train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=prefetched_val_ds,
)

with open('1e-2_manyGpuAug_stride2_batch64_128filters-200e/training_history.pkl', 'wb') as file:
    pickle.dump(history.history, file)
    
# summarize history for accuracy
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('1e-2_manyGpuAug_stride2_batch64_128filters-200e/accuracy_plot.png')
# summarize history for loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('1e-2_manyGpuAug_stride2_batch64_128filters-200e/loss_plot.png')

In [ ]:
# Replot training data

import matplotlib.pyplot as plt
import pickle

# Load the training history
with open('1e-3_manyGpuAug_stride2_batch64_128filters-ended/training_history.pkl', 'rb') as file:
    history2 = pickle.load(file)

# Retrieve accuracy and loss history
accuracy = history2['accuracy']
loss = history2['loss']
val_accuracy = history2['val_accuracy']
val_loss = history2['val_loss']

# Plot accuracy
plt.figure(figsize=(10, 5))
plt.plot(accuracy)
plt.plot(val_accuracy)
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim(0,1)

# Plot loss
plt.figure(figsize=(10, 5))
plt.plot(loss)
plt.plot(val_loss)
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')


In [ ]:
model = keras.models.load_model("model_1.keras")
image_size = (256, 256)
batch_size = 128

In [ ]:
# Inference
import numpy as np

img = keras.utils.load_img(
    "dataset/without_augmentation/Pepper___bell_healthy/image (5).JPG", target_size=image_size
)
img_array = keras.utils.img_to_array(img)

print("Img array")
print(img_array)

img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
print(predictions[0])

#class_index = np.argmax(predictions[0])
#class_name = class_names[class_index] 

k = 5
top_classes = np.argsort(predictions[0])[::-1][:k]  # Get indices of top k classes
top_scores = predictions[0][top_classes]

for i in range(k):
    class_index = top_classes[i]
    confidence = top_scores[i]
    class_name = class_names[class_index]
    print(f"Class {class_name}: Confidence = {confidence:.2%}")
#score = float(predictions[0])
#print(f"The score of the image is {score}.")

In [ ]:
model.save("model_19e_10-2_adam.keras")

In [ ]:
# Evaluation of test_dataset

saved_model_path = "3datasets_1e-3_manyGpuAug_stride2_batch64_64filters_moreFilters_categorical/100.keras"
model = tf.keras.models.load_model(saved_model_path)

test_loss, test_accuracy = model.evaluate(val_ds)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [2]:
#K-fold complete

import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

k = 4
folds = [0,1]

for i in folds:
    print(f'Fold: {i}')
    
    # Using k-fold cross-validation

    image_size = (256, 256)
    batch_size = 64

    ds = tf.keras.utils.image_dataset_from_directory(
        "dataset/without_augmentation",
        image_size=image_size,
        batch_size=batch_size,
        label_mode='categorical',
        seed=1337
    )

    # Split the dataset into k folds using tf.data.Dataset methods
    dataset_size = len(ds)
    print('Dataset size')
    print(dataset_size)
    fold_size = dataset_size // k
    print('Fold size')
    print(fold_size)

    start = i * fold_size
    print(start)
    end = (i + 1) * fold_size
    print(end)

    # Create training and validation datasets for the current fold
    val_ds = ds.skip(start).take(fold_size)
    train_ds = ds.take(start).concatenate(ds.skip(end))

    print('Val size')
    print(val_ds.cardinality())
    print('Train size')
    print(train_ds.cardinality())

    # Data augmentation definition

    data_augmentation = keras.Sequential(
        [
            layers.RandomFlip("horizontal_and_vertical"),
            layers.RandomBrightness(0.3),
            layers.RandomZoom(0.4),
            layers.RandomContrast(0.3),
        ]
    )

    # GPU-augmented - Prefetching samples in GPU memory helps maximize GPU utilization.

    data_augmentation_test = keras.Sequential(
        [
           layers.RandomRotation(0), 
        ]
    )

    train_ds = train_ds.map(
        lambda img, label: (data_augmentation_test(img), label),
        num_parallel_calls=tf.data.AUTOTUNE,
    )

    prefetched_train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
    prefetched_val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

    def make_model(input_shape, num_classes):
        inputs = keras.Input(shape=input_shape)
        # x = data_augmentation(inputs)

        # Entry block
        # x = layers.Rescaling(1.0 / 255)(x)
        x = layers.Rescaling(1.0 / 255)(inputs)
        x = layers.Conv2D(64, 3, strides=2, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)

        previous_block_activation = x  # Set aside residual

        for size in [128, 256, 512]:
            x = layers.Activation("relu")(x)
            x = layers.SeparableConv2D(size, 3, padding="same")(x)
            x = layers.BatchNormalization()(x)

            x = layers.Activation("relu")(x)
            x = layers.SeparableConv2D(size, 3, padding="same")(x)
            x = layers.BatchNormalization()(x)

            x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

            # Project residual
            residual = layers.Conv2D(size, 1, strides=2, padding="same")(
                previous_block_activation
            )
            x = layers.add([x, residual])  # Add back residual
            previous_block_activation = x  # Set aside next residual

        x = layers.SeparableConv2D(1024, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)

        x = layers.GlobalAveragePooling2D()(x)

        activation = "softmax"
        units = num_classes


        x = layers.Dropout(0.5)(x)
        outputs = layers.Dense(units, activation=activation)(x)
        return keras.Model(inputs, outputs)


    model = make_model(input_shape=image_size + (3,), num_classes=39)
    keras.utils.plot_model(model, show_shapes=True)

    # Normal training

    epochs = 100
    print(f"De novo: {i}")

    callbacks = [
        #tf.keras.callbacks.History(),
        keras.callbacks.ModelCheckpoint(f'fold_{i}' + "_1e-2_manyGpuAug_stride2_batch64_64filters_categorical/{epoch}.keras")
    ]
    model.compile(
        optimizer=keras.optimizers.Adam(1e-2),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    history = model.fit(
        prefetched_train_ds,
        epochs=epochs,
        callbacks=callbacks,
        validation_data=prefetched_val_ds,
    )

    with open(f'fold_{i}_1e-2_manyGpuAug_stride2_batch64_64filters_categorical/training_history.pkl', 'wb') as file:
        pickle.dump(history.history, file)

    # summarize history for accuracy
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig(f'fold_{i}_1e-2_manyGpuAug_stride2_batch64_64filters_categorical/accuracy_plot.png')
    # summarize history for loss
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig(f'fold_{i}_1e-2_manyGpuAug_stride2_batch64_64filters_categorical/loss_plot.png')
    
    val_accuracy = model.evaluate(val_ds)

    with open(f'fold_{i}_1e-2_manyGpuAug_stride2_batch64_64filters_categorical/result', 'w') as file:
        # Write the variable value to the file
        file.write(str(val_accuracy))


Fold: 0
Found 55447 files belonging to 39 classes.


2023-07-09 18:44:22.709572: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-09 18:44:22.731976: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-09 18:44:22.732225: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-09 18:44:22.734392: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-09 18:44:22.734495: I tensorflow/compile

Dataset size
867
Fold size
216
0
216
Val size
tf.Tensor(216, shape=(), dtype=int64)
Train size
tf.Tensor(651, shape=(), dtype=int64)
De novo: 0
Epoch 1/100


2023-07-09 18:44:24.688830: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [55447]
	 [[{{node Placeholder/_0}}]]
2023-07-09 18:44:24.689131: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_25' with dtype int32 and shape [55447]
	 [[{{node Placeholder/_25}}]]
2023-07-09 18:44:31.108444: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-07-09 18:44:33.265098: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-07-

651/651 [==============================] - ETA: 0s - loss: 1.6253 - accuracy: 0.5493

2023-07-09 18:45:51.666788: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [55447]
	 [[{{node Placeholder/_0}}]]
2023-07-09 18:45:51.667264: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [55447]
	 [[{{node Placeholder/_4}}]]


651/651 [==============================] - 97s 128ms/step - loss: 1.6253 - accuracy: 0.5493 - val_loss: 5.2826 - val_accuracy: 0.3348
Epoch 2/100
651/651 [==============================] - 84s 126ms/step - loss: 0.7883 - accuracy: 0.7625 - val_loss: 0.8106 - val_accuracy: 0.7772
Epoch 3/100
651/651 [==============================] - 84s 126ms/step - loss: 0.4870 - accuracy: 0.8485 - val_loss: 2.9958 - val_accuracy: 0.5339
Epoch 4/100
651/651 [==============================] - 85s 126ms/step - loss: 0.3557 - accuracy: 0.8885 - val_loss: 3.5254 - val_accuracy: 0.4855
Epoch 5/100
651/651 [==============================] - 85s 127ms/step - loss: 0.2723 - accuracy: 0.9131 - val_loss: 1.0023 - val_accuracy: 0.7595
Epoch 6/100
651/651 [==============================] - 84s 126ms/step - loss: 0.2340 - accuracy: 0.9261 - val_loss: 1.3421 - val_accuracy: 0.6929
Epoch 7/100
651/651 [==============================] - 85s 127ms/step - loss: 0.1973 - accuracy: 0.9387 - val_loss: 0.2881 - val_accurac